In [1]:
import requests
import pandas as pd
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import censusdata
import numpy as np
import bokeh
from census import Census
import us
from ipywidgets import interact, interact_manual, Dropdown, IntSlider
import seaborn as sns
from bokeh.plotting import ColumnDataSource, figure, show, output_file
from bokeh.layouts import gridplot, column, row, WidgetBox
from bokeh.models import Panel, CustomJS, Div, Select
from ipywidgets import interact, interact_manual, Dropdown
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [2]:
# path = r'C:/Users/ShrekTheOger/Documents/GitHub/Data-II-Project'
# path = r'/Users/bowenli/Documents/GitHub/Data-II-Project'
path = r'C:\Users\engel\Documents\GitHub\Data-II-Project'

In [3]:
# panda table setting 
# https://re-thought.com/how-to-suppress-scientific-notation-in-pandas/
# pd.set_option('display.expand_frame_repr', False)
# pd.set_option('display.precision', 2)

# set float display
pd.options.display.float_format = '{:.0f}'.format

# Reading US Census data using API -- Population  + Income 
population data has year 2010 to 2019, and per capital income from year 2010, 2015, 2019

In [4]:
# references 
# https://towardsdatascience.com/mapping-us-census-data-with-python-607df3de4b9c
# https://pypi.org/project/CensusData/
# https://towardsdatascience.com/using-the-us-census-api-for-data-analysis-a-beginners-guide-98063791785c

# function for retrieving population data
def get_acs5_county_population(year):
    data = censusdata.download('acs5', year, censusdata.censusgeo([('county', '*')]),
                               ['B05002_001E', 'B05002_002E', 'B05002_003E', 'B05002_004E', 'B05002_009E', 'B05002_013E'])
    
    # create dictionary for the column names
    column_names = {'B05002_001E': 'total_population', 'B05002_002E': 'total_native', 
                    'B05002_003E': 'total_born_in_state', 'B05002_004E': 'total_born_out_state',
                    'B05002_009E': 'total_born_outside_US', 'B05002_013E': 'total_foreign_born'}
    
    # create new column for county name and state name, the index would be FIPS codes 
    new_indices = []
    county_names = []
    county_ids = []
    state_names = []
    state_ids = []
    
    
    for index in data.index.tolist():
        new_index = index.geo[0][1] + index.geo[1][1]
        new_indices.append(new_index)
        county_name = index.name.split(',')[0]
        county_names.append(county_name)
        state_name = index.name.split(',')[1]
        state_names.append(state_name)
        state_id = index.geo[0][1]
        state_ids.append(state_id)
        county_id = index.geo[1][1]
        county_ids.append(county_id)

    data.index = new_indices
    data['county_name'] = county_names
    data['state_name'] = state_names
    data['county_id'] = county_ids
    data['state_id'] =  state_ids
    data['year'] =  year
    
    # replace column names 
    data = data[['year', 'state_name', 'county_name', 'state_id', 'county_id', 'B05002_001E',
                 'B05002_002E', 'B05002_003E', 'B05002_004E', 'B05002_009E', 'B05002_013E']]
    data = data.rename(column_names, axis=1) 
    data['state_name'] = data['state_name'].str.strip()
    
    return data

In [5]:
# function for retrieving income data
def get_acs5_county_income(year):
    data = censusdata.download('acs5', year, censusdata.censusgeo([('county', '*')]), ['B19301_001E'])
    
    # create dictionary for the column names
    column_names = {'B19301_001E': 'income_past12m'}
    
    # create new column for county name and state name, the index would be FIPS codes 
    new_indices = []
    county_names = []
    county_ids = []
    state_names = []
    state_ids = []
    
    for index in data.index.tolist():
        new_index = index.geo[0][1] + index.geo[1][1]
        new_indices.append(new_index)
        county_name = index.name.split(',')[0]
        county_names.append(county_name)
        state_name = index.name.split(',')[1]
        state_names.append(state_name)
        state_id = index.geo[0][1]
        state_ids.append(state_id)
        county_id = index.geo[1][1]
        county_ids.append(county_id)

    data.index = new_indices
    data['county_name'] = county_names
    data['state_name'] = state_names
    data['county_id'] = county_ids
    data['state_id'] =  state_ids
    data['year'] =  year
    
    # replace column names 
    data = data[['year', 'state_name', 'county_name', 'state_id', 'county_id', 'B19301_001E']]
    data = data.rename(column_names, axis=1)
    
    data['state_name'] = data['state_name'].str.strip()
    
    return data

In [6]:
def merge_population_df(ystart, yend):
    years = list(range(ystart, yend))
    df_population = pd.DataFrame()
    
    #retriving census data from year start to year end and merge into one dataframe
    for year in years: 
        population_year = get_acs5_county_population(year) 
        df_population = df_population.append(population_year)
    
    df_population.reset_index(drop = True, inplace = True)
    
    return df_population

In [7]:
pop_df = merge_population_df(2010, 2020)
pop_df

,year,state_name,county_name,state_id,county_id,total_population,total_native,total_born_in_state,total_born_out_state,total_born_outside_US,total_foreign_born
0,2010,Puerto Rico,Las Marías Municipio,72,083,10156,10087,0,360,9727,69
1,2010,Puerto Rico,San Germán Municipio,72,125,35997,35723,0,1802,33921,274
2,2010,Puerto Rico,Comerío Municipio,72,045,20773,20755,0,375,20380,18
3,2010,Puerto Rico,Canóvanas Municipio,72,029,47151,45970,0,2096,43874,1181
4,2010,Puerto Rico,Rincón Municipio,72,117,15203,15099,0,1617,13482,104
...,...,...,...,...,...,...,...,...,...,...,...
32199,2019,Tennessee,Crockett County,47,033,14399,13774,11231,2490,53,625
32200,2019,Tennessee,Lake County,47,095,7401,7307,5436,1839,32,94
32201,2019,Tennessee,Knox County,47,093,461104,438362,272843,161946,3573,22742
32202,2019,Washington,Benton County,53,005,197518,176385,104182,70549,1654,21133


In [8]:
def merge_income_df(ystart, yend):
    years = list(range(ystart, yend))
    df_income = pd.DataFrame()

    #retriving census data on per capital income from year 2020 to 2019 
    #(conduct survey every 5 years) into one dataframe
    for year in years: 
        income_year = get_acs5_county_income(year) 
        df_income = df_income.append(income_year)
    
    
    df_income.reset_index(drop = True, inplace = True)
    
    return df_income

In [9]:
income_df = merge_income_df(2010, 2020)
income_df

,year,state_name,county_name,state_id,county_id,income_past12m
0,2010,Puerto Rico,Las Marías Municipio,72,083,6417
1,2010,Puerto Rico,San Germán Municipio,72,125,8066
2,2010,Puerto Rico,Comerío Municipio,72,045,6755
3,2010,Puerto Rico,Canóvanas Municipio,72,029,9852
4,2010,Puerto Rico,Rincón Municipio,72,117,8768
...,...,...,...,...,...,...
32199,2019,Tennessee,Crockett County,47,033,23771
32200,2019,Tennessee,Lake County,47,095,15732
32201,2019,Tennessee,Knox County,47,093,33229
32202,2019,Washington,Benton County,53,005,32882


# Reading US Universities annual Fund amount from US Gov

In [10]:
# Read Universities R&D Data based on ID 
# # https://stackoverflow.com/questions/45416684/python-pandas-replace-multiple-columns-zero-to-nan
rd_id = os.path.join(path, 'NCSES', 'HERD_data_IPEDS.csv')
rd_id_data = pd.read_csv(rd_id, skiprows=10)


rd_id_data = rd_id_data.iloc[2:]
column_names_id = {'Unnamed: 0': 'state_name', '<Fiscal Year>': 'IPEDSID'}
rd_id_data = rd_id_data.rename(column_names_id, axis=1) 
rd_id_data = rd_id_data[rd_id_data['IPEDSID'].str.contains('Total for selected values') == False]
rd_id_data = rd_id_data[rd_id_data['IPEDSID'].str.contains('No match or exact match for IPEDS UnitID') == False]
rd_id_data.iloc[:,2:] = rd_id_data.iloc[:,2:].replace({'-':np.nan}).replace(r',','',regex=True)
rd_id_data.iloc[:,2:] = rd_id_data.iloc[:,2:].astype(float)
rd_id_data  = rd_id_data.sort_values(['2010'], ascending = False)

rd_id_data = rd_id_data.head(100)

rd_id_data = rd_id_data.reset_index(drop = True)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\engel\\Documents\\GitHub\\Data-II-Project\\NCSES\\HERD_data_IPEDS.csv'

In [ ]:
years = list(range(2010, 2020))
years_string = [str(year) for year in years]

rd_id_data_filter = rd_id_data[['state_name', 'IPEDSID', *years_string]]
rd_id_df = rd_id_data_filter.melt(id_vars=["state_name", "IPEDSID"], 
                                  var_name="Year",value_name="Fund")
rd_id_df

## Create Line Plot 

In [ ]:
# function to generate color
def color_gen(n):
    palette = bokeh.palettes.cividis(n)
    for color in palette:
        yield color

In [ ]:
def filter_state_county(df, state, county): 
    df['state_name'].str.lstrip()
    data = df[(df['state_name'] == state) & (df['county_name'] == county)]
    return data

In [ ]:
population_filter = filter_state_county(pop_df, 'Illinois', 'Cook County').copy()

In [ ]:
population_filter

In [ ]:
population_filter.reset_index()

In [ ]:
pop_variables = ['total_born_in_state', 'total_born_out_state', 'total_born_outside_US', 'total_foreign_born']
get_color = color_gen(len(pop_variables))

In [ ]:
population_filter['share_in_state'] = (population_filter['total_born_in_state']/population_filter['total_population'])
population_filter

In [ ]:
def plot_stackedbar(df, variables, title):
    #data = df
    data = pd.DataFrame()
    data['year'] = df['year']
    data['total_born_in_state'] = (df['total_born_in_state']/df['total_population'])
    data['total_born_out_state'] = (df['total_born_out_state']/df['total_population'])
    data['total_born_outside_US'] = (df['total_born_outside_US']/df['total_population'])
    data['total_foreign_born'] = (df['total_foreign_born']/df['total_population'])
    selected_data = data[['year', *variables]]
    selected_data = selected_data.sort_values('year', ascending = True).reset_index(drop=True)
    
    selected_data.plot(x='year', kind='bar', figsize=(12, 10), stacked=True, color=get_color)
    plt.legend(title='category', bbox_to_anchor=(1.3,0.5), loc='center right')
    plt.ylim(0.0, 1.0)
    plt.title(title, fontsize=20)
    plt.xticks(rotation=0, fontsize=12)
    plt.ylabel('natural units', fontsize=15)
    plt.xlabel('year', fontsize=15)
    
    return plt

In [ ]:
plot_stackedbar(population_filter, pop_variables, 'Share of Population');

In [ ]:
source = ColumnDataSource(data=population_filter)
    
tool = [('value', '@'+'total_population'), ('year','@year')]

plot = figure(title='Merced County', x_axis_label='year', y_axis_label='natural units', tooltips=tool, plot_height=400)
    
plot.line(x='year', y='total_population', source=source, color='blue')
plot.circle(x='year', y='total_population', source=source, color='grey')

show(plot)

# Read Shape file 
Retrive shape file of _County level_ US map and geometries of US Universities that ranked top 100 on Fund list

## county map refine

In [ ]:
# https://www.census.gov/geographies/mapping-files/time-series/geo/cartographic-boundary.html
county_shp = os.path.join(path, 'cb_2020_us_county_20m', 'cb_2020_us_county_20m.shp')
county = gpd.read_file(county_shp)

In [ ]:
# making the plot including only 48 states within NA
contiguous_fips = [state.fips for state in us.STATES_CONTIGUOUS]
mask_fips = county['STATEFP'].isin(contiguous_fips)
county_48 = county[mask_fips]

In [ ]:
county_48

## US Uni refine

In [ ]:
universities_shp = os.path.join(path, 'Colleges_and_Universities-shp', 'Colleges_and_Universities.shp')
universities = gpd.read_file(universities_shp)

In [ ]:
universities.insert(6,'fips', universities['STATE'].map(us.states.mapping('abbr','fips')))
mask_contiguous_universities = universities['fips'].isin(contiguous_fips)
uni_48 = universities[mask_contiguous_universities]

In [ ]:
uni_48.plot();

In [ ]:
uni_top = uni_48.merge(rd_id_df, how = 'inner', on = 'IPEDSID')
uni_top = uni_top.to_crs(county_48.crs)
uni_top.plot();

# Ipywidget for final presentation
Here we have three attributes -- state, [population / per capital income], and year

In [ ]:
us_contiguous = [n.name for n in us.STATES_CONTIGUOUS]

In [ ]:
#Get the state level county map and state level uni location
def county_uni_map(State):
    state_county_map = county_48[county_48['STATE_NAME'] == State]
    state_uni_map = uni_top[uni_top.state_name == State]
    return state_county_map, state_uni_map

In [ ]:
# Get the state level income per capita and population in certain year
def county_info(State, Year, Indicator):
    if Indicator == 'Population Growth':
        county_pop = pop_df.loc[(pop_df['state_name'] == State) & (pop_df['year'] == Year)]
        choice = county_pop
    else:
        county_income = income_df.loc[(income_df['state_name'] == State) & (income_df['year'] == Year)]
        choice = county_income
    
    return choice

In [ ]:
# create widget options

#options = {'North America':['Unemployment Rate', 'GDP per Capita'], 
           #'USA':['Unemployment Rate', 'Personal Income per Capita']}

# create widget
first_widget = Dropdown(options=us_contiguous)
second_widget = Dropdown(options=['Personal Income per Capita','Population Growth', ])
third_widget= IntSlider(value=2010, min=2010, max=2019, step=1)

#def update(*args):
   # second_widget.options = options[first_widget.value]   
#first_widget.observe(update)
 
                           
def plot_area(State, Indicator, Year):
    #https://jcutrer.com/python/learn-geopandas-plotting-usmaps
    state_map, uni_map = county_uni_map(State)
    indicator_df = county_info(State, Year, Indicator)
    
    #Combine map data with info data
    df = state_map.merge(indicator_df, how = 'inner', left_on = 'COUNTYFP', right_on ='county_id')
    
    fig, ax = plt.subplots(figsize=(20,20))
    df.apply(lambda x: ax.annotate(s=x.NAME, xy=x.geometry.centroid.coords[0], 
                                   ha='center', fontsize=12),axis=1)
    df.boundary.plot(ax=ax, color='Black', linewidth=2)
    df.plot(ax=ax,column = 'income_past12m', figsize=(12, 12),
            legend = True, cmap='OrRd',
            legend_kwds={'label': "undetermined", 
                         'orientation': "vertical"})
    
    ax.text(-0.05, 0.5,'undetermined' , transform=ax.transAxes,
            fontsize=50, color='black', alpha=0.8,
            ha='center', va='center', rotation='90')
    
    uni_map.apply(lambda x: ax.annotate(s="Fund amount: " + str(x['Fund']), 
                                        xy= (x.geometry.centroid.coords[0][0]+0.01, 
                                             x.geometry.centroid.coords[0][1]-0.007),
                                        ha='left', color='#000077', # blue
                                        fontsize=10),axis=1)
    uni_map.plot(ax=ax, marker='*', color='green', markersize=600)
    
    ax.axis('off'); 
        
interact(plot_area, State=first_widget, Indicator=second_widget, Year=third_widget);

## Create A Big Dataframe 

In [ ]:
uni_top_id = uni_top['IPEDSID'].unique()

In [ ]:
#https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values
universities_filter = universities.loc[universities['IPEDSID'].isin(uni_top_id)].copy()

universities_filter = universities_filter[['IPEDSID', 'NAME', 'STATE', 'COUNTY', 'COUNTYFIPS', 'geometry']]

In [ ]:
universities_filter